In [1]:
import pandas as pd
import numpy as np

from glob import glob
import re
import string
import funcy as fp
from gensim import models
from gensim.corpora import Dictionary, MmCorpus
import nltk

In [2]:
clean_mini = pd.read_csv('clean_python_stack_052620.csv')

In [3]:
clean_mini.head()

,question,summary,tags,user_id,no_of_votes,no_of_answers,date,clean_text
0,multiple json objects into a list,"{""Id"": 1,""product"": ""Mobile"",""price"": 32889,""d...","['python', 'json']",diveinsky,0.0,1.0,2019-08-02,multiple json object list
1,cannot read in image in colour in opencv python,I have just started to use Opencv using python...,"['python-2.7', 'opencv']",Yogesh,0.0,2.0,2019-08-02,can not read image colour opencv python
2,How to resolve related foreign key at insert t...,Motivations\n\nI have data coming from a sourc...,"['python', 'pandas', 'orm', 'sqlalchemy', 'for...",jlandercy,4.0,2.0,2019-08-02,resolve relate foreign key insert time use sql...
3,syntax error in python shorthand expression,I am using bs4 to scrape a website I have thi...,"['python', 'python-3.x', 'beautifulsoup']",QHarr,2.0,2.0,2019-08-02,syntax error python shorthand expression
4,When do we need to pip install future [duplicate],As per the documentation the module future pro...,"['python', 'future']",Jay Joshi,0.0,0.0,2019-08-02,need pip install future duplicate


In [4]:
clean_mini.shape

(1464969, 8)

In [5]:
clean_mini['clean_text'].isna().sum()

12

In [6]:
clean_mini = clean_mini.dropna(subset=['clean_text'])

In [7]:
clean_mini['clean_text'] = clean_mini['clean_text'].str.split(' ')

In [8]:
clean_mini['clean_text'].head()

0                       [multiple, json, object, list]
1      [can, not, read, image, colour, opencv, python]
2    [resolve, relate, foreign, key, insert, time, ...
3       [syntax, error, python, shorthand, expression]
4              [need, pip, install, future, duplicate]
Name: clean_text, dtype: object

In [9]:
def nltk_stopwords():
    return set(nltk.corpus.stopwords.words('english'))

def prep_corpus(docs, additional_stopwords=set(), no_below=5, no_above=0.5):
  print('Building dictionary...')
  dictionary = Dictionary(docs)
  stopwords = nltk_stopwords().union(additional_stopwords)
  stopword_ids = map(dictionary.token2id.get, stopwords)
  dictionary.filter_tokens(stopword_ids)
  dictionary.compactify()
  dictionary.filter_extremes(no_below=no_below, no_above=no_above, keep_n=None)
  dictionary.compactify()

  print('Building corpus...')
  corpus = [dictionary.doc2bow(doc) for doc in docs]

  return dictionary, corpus

In [10]:
dictionary, corpus = prep_corpus(clean_mini['clean_text'])

Building dictionary...
Building corpus...


In [11]:
MmCorpus.serialize('models/python_corpus.mm', corpus)
dictionary.save('models/python_dict.dict')

In [12]:
%%time
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=15)
                                      
lda.save('models/python_lda.model')

CPU times: user 14min 6s, sys: 9.83 s, total: 14min 16s
Wall time: 6min 39s


In [13]:
#LdaModel.load('models/python_lda.model')

In [14]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis

In [15]:
%%time
vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

CPU times: user 45min 1s, sys: 51.1 s, total: 45min 52s
Wall time: 46min 46s


In [16]:
%%time
p = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.save_html(p, 'python_lda.html')

CPU times: user 39min 21s, sys: 25.2 s, total: 39min 46s
Wall time: 40min 3s


In [ ]:
#get_document_topics(bow, minimum_probability=None, minimum_phi_value=None, per_word_topics=False)

In [23]:
from gensim.models import LdaModel

In [33]:
doc_topics = lda.get_document_topics(bow = corpus[0])

In [34]:
doc_topics

[(0, 0.013333494),
 (1, 0.0133335125),
 (2, 0.213332),
 (3, 0.21333477),
 (4, 0.013333494),
 (5, 0.013333494),
 (6, 0.013333494),
 (7, 0.013333494),
 (8, 0.21333066),
 (9, 0.013333494),
 (10, 0.013333513),
 (11, 0.013333494),
 (12, 0.21333407),
 (13, 0.013333494),
 (14, 0.013333511)]

In [84]:
def topic_found(bow):
    
    doc_topics = lda.get_document_topics(bow)
    
    topic,values = map(list,zip(*doc_topics))

    values_iter = iter(values)
    
    filled_values = []
    
    for e in range(15):
    
        if e in topic: 
            filled_values.append(next(values_iter))
        
        else:
            filled_values.append(0.00)  
           
    return filled_values

In [81]:
def topic_find(bow):
    
    doc_topics = lda.get_document_topics(bow)
    
    topic,values = zip(*doc_topics)
            
    return topic, values

In [90]:
%%time
topic_values = pd.DataFrame([topic_found(x) for x in corpus]) 

CPU times: user 6min 32s, sys: 1.92 s, total: 6min 34s
Wall time: 6min 33s


In [91]:
topic_values.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.013333,0.013334,0.213332,0.213335,0.013333,0.013333,0.013333,0.013333,0.213331,0.013333,0.013334,0.013333,0.213334,0.013333,0.013334
1,0.011112,0.011112,0.011112,0.011112,0.011112,0.429767,0.011112,0.011112,0.011112,0.011112,0.011112,0.011112,0.011112,0.011112,0.425783
2,0.131271,0.136112,0.000000,0.000000,0.000000,0.000000,0.000000,0.399667,0.000000,0.000000,0.122922,0.000000,0.000000,0.000000,0.135944
3,0.547031,0.013334,0.013334,0.013334,0.013334,0.013334,0.013334,0.013334,0.013334,0.013334,0.013334,0.279633,0.013334,0.013334,0.013334
4,0.011112,0.011112,0.344460,0.011112,0.011112,0.229511,0.011112,0.011112,0.011112,0.011112,0.011112,0.292687,0.011112,0.011112,0.011112


In [92]:
python_topics = pd.concat([clean_mini, topic_values],axis=1)

In [93]:
python_topics.head()

,question,summary,tags,user_id,no_of_votes,no_of_answers,date,clean_text,0,1,...,5,6,7,8,9,10,11,12,13,14
0,multiple json objects into a list,"{""Id"": 1,""product"": ""Mobile"",""price"": 32889,""d...","['python', 'json']",diveinsky,0.0,1.0,2019-08-02,"[multiple, json, object, list]",0.013333,0.013334,...,0.013333,0.013333,0.013333,0.213331,0.013333,0.013334,0.013333,0.213334,0.013333,0.013334
1,cannot read in image in colour in opencv python,I have just started to use Opencv using python...,"['python-2.7', 'opencv']",Yogesh,0.0,2.0,2019-08-02,"[can, not, read, image, colour, opencv, python]",0.011112,0.011112,...,0.429767,0.011112,0.011112,0.011112,0.011112,0.011112,0.011112,0.011112,0.011112,0.425783
2,How to resolve related foreign key at insert t...,Motivations\n\nI have data coming from a sourc...,"['python', 'pandas', 'orm', 'sqlalchemy', 'for...",jlandercy,4.0,2.0,2019-08-02,"[resolve, relate, foreign, key, insert, time, ...",0.131271,0.136112,...,0.000000,0.000000,0.399667,0.000000,0.000000,0.122922,0.000000,0.000000,0.000000,0.135944
3,syntax error in python shorthand expression,I am using bs4 to scrape a website I have thi...,"['python', 'python-3.x', 'beautifulsoup']",QHarr,2.0,2.0,2019-08-02,"[syntax, error, python, shorthand, expression]",0.547031,0.013334,...,0.013334,0.013334,0.013334,0.013334,0.013334,0.013334,0.279633,0.013334,0.013334,0.013334
4,When do we need to pip install future [duplicate],As per the documentation the module future pro...,"['python', 'future']",Jay Joshi,0.0,0.0,2019-08-02,"[need, pip, install, future, duplicate]",0.011112,0.011112,...,0.229511,0.011112,0.011112,0.011112,0.011112,0.011112,0.292687,0.011112,0.011112,0.011112
